In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from secret import username, password

In [2]:
# Study data files
imdb_movies_filepath = "data/IMDb movies.csv"
imdb_names_filepath = "data/IMDb names.csv"
imdb_ratings_filepath = "data/IMDb ratings.csv"

In [3]:
# Read the data into Pandas DataFrame
imdb_movies = pd.read_csv(imdb_movies_filepath)
imdb_names = pd.read_csv(imdb_names_filepath)
imdb_ratings = pd.read_csv(imdb_ratings_filepath)

# PERSON FILE

In [4]:
#Look at imported names CSV
imdb_names.head()

,imdb_name_id,name,birth_name,height,bio,birth_details,birth_year,date_of_birth,place_of_birth,death_details,death_year,date_of_death,place_of_death,reason_of_death,spouses,divorces,spouses_with_children,children,primary_profession,known_for_titles
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899.0,1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987.0,1987-06-22,"Los Angeles, California, USA",pneumonia,2,0,1,2,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0043044"
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924.0,1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014.0,2014-08-12,"New York City, New York, USA",stroke,2,1,2,3,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934.0,1934-09-28,"Paris, France",NaN,NaN,NaN,NaN,NaN,4,3,1,1,"actress,soundtrack,producer","tt0054452,tt0059956,tt0057345,tt0049189"
3,nm0000004,John Belushi,John Adam Belushi,173.0,"John Belushi was born in Chicago, Illinois, US...","January 24, 1949 in Chicago, Illinois, USA",1949.0,1949-01-24,"Chicago, Illinois, USA","March 5, 1982 in Hollywood, Los Angeles, Calif...",1982.0,1982-03-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication,1,0,0,0,"actor,writer,soundtrack","tt0078723,tt0072562,tt0080455,tt0077975"
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179.0,"Ernst Ingmar Bergman was born July 14, 1918, t...","July 14, 1918 in Uppsala, Uppsala län, Sweden",1918.0,1918-07-14,"Uppsala, Uppsala län, Sweden","July 30, 2007 in Fårö, Gotlands län, Sweden",2007.0,2007-07-30,"Fårö, Gotlands län, Sweden",NaN,5,4,5,8,"writer,director,actor","tt0050976,tt0083922,tt0069467,tt0050986"


## person table

In [5]:
# Return only wanted columns for person table
name_columns = ['imdb_name_id', 'name', 'birth_name', 'birth_year', 'death_year', 'height']
new_name_df = imdb_names[name_columns].copy()

# Rename the column headers to match ERD
transform_name_df = new_name_df.rename(columns={'imdb_name_id': 'person_id',
                                                 'name': 'person_name'})

# Drop duplicate people with same name
final_name = transform_name_df.drop_duplicates('person_name')

final_name.head()

,person_id,person_name,birth_name,birth_year,death_year,height
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,1899.0,1987.0,177.0
1,nm0000002,Lauren Bacall,Betty Joan Perske,1924.0,2014.0,174.0
2,nm0000003,Brigitte Bardot,Brigitte Bardot,1934.0,NaN,166.0
3,nm0000004,John Belushi,John Adam Belushi,1949.0,1982.0,173.0
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,1918.0,2007.0,179.0


In [6]:
#Inspect name table for column type
final_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173127 entries, 0 to 175714
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   person_id    173127 non-null  object 
 1   person_name  173127 non-null  object 
 2   birth_name   173127 non-null  object 
 3   birth_year   74487 non-null   float64
 4   death_year   26639 non-null   float64
 5   height       29784 non-null   float64
dtypes: float64(3), object(3)
memory usage: 9.2+ MB


## job_title table

In [7]:
#Create lists for job title id and job title
job_title_id = ['jt001', 'jt002', 'jt003']
job_title = ['actor', 'director', 'writer']

#Convert list to DataFrame
job_title_df = pd.DataFrame(list(zip(job_title_id, job_title)), 
               columns =['job_title_id', 'job_title'])
job_title_df.head()

,job_title_id,job_title
0,jt001,actor
1,jt002,director
2,jt003,writer


# MOVIES FILE

In [8]:
# Inspect movies CSV
imdb_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,NaN,24.0,3.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,NaN,28.0,14.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,NaN,12.0,5.0


## movie table

In [9]:
# Return wanted columns for movie table
movie_columns = ['imdb_title_id', 'title', 'year', 'duration', 'description', 'votes', 'avg_vote']
new_movie_df = imdb_movies[movie_columns].copy()
new_movie_df.head()

,imdb_title_id,title,year,duration,description,votes,avg_vote
0,tt0000574,The Story of the Kelly Gang,1906,70,True story of notorious Australian outlaw Ned ...,537,6.1
1,tt0001892,Den sorte drøm,1911,53,Two men of high rank are both wooing the beaut...,171,5.9
2,tt0002101,Cleopatra,1912,100,The fabled queen of Egypt's affair with Roman ...,420,5.2
3,tt0002130,L'Inferno,1911,68,Loosely adapted from Dante's Divine Comedy and...,2019,7.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,60,"An account of the life of Jesus Christ, based ...",438,5.7


In [10]:
# Rename the column headers
transform_movie_df = new_movie_df.rename(columns={'imdb_title_id': 'movie_id',
                                                 'title': 'movie_title',
                                                 'year': 'year_published',
                                                 'duration': 'movie_duration',
                                                 'votes': 'votes_count',
                                                 'avg_vote': 'votes_avg'})

# Clean the data by dropping duplicates and setting the index
transform_movie_df.drop_duplicates("movie_id", inplace=True)
transform_movie_df.set_index("movie_id", inplace=True)

transform_movie_df.head()

,movie_title,year_published,movie_duration,description,votes_count,votes_avg
movie_id,,,,,,
tt0000574,The Story of the Kelly Gang,1906,70,True story of notorious Australian outlaw Ned ...,537,6.1
tt0001892,Den sorte drøm,1911,53,Two men of high rank are both wooing the beaut...,171,5.9
tt0002101,Cleopatra,1912,100,The fabled queen of Egypt's affair with Roman ...,420,5.2
tt0002130,L'Inferno,1911,68,Loosely adapted from Dante's Divine Comedy and...,2019,7.0
tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,60,"An account of the life of Jesus Christ, based ...",438,5.7


## Directors

In [11]:
# Return only directors and movie title id's
director_df = imdb_movies[['imdb_title_id', 'director']].copy()
director_df.head()

,imdb_title_id,director
0,tt0000574,Charles Tait
1,tt0001892,Urban Gad
2,tt0002101,Charles L. Gaskill
3,tt0002130,"Francesco Bertolini, Adolfo Padovan"
4,tt0002199,Sidney Olcott


In [12]:
# Rename the column headers
transform_director_df = director_df.rename(columns={'imdb_title_id': 'movie_id',
                                                     'director': 'person_name'})

# Split person_name column by comma and insert it as new row
new_director_df = transform_director_df.assign(person_name=transform_director_df.person_name.str.split(', ')).explode('person_name')

# Add job title id to all rows
new_director_df['job_title_id'] ='jt002'
new_director_df.head()

,movie_id,person_name,job_title_id
0,tt0000574,Charles Tait,jt002
1,tt0001892,Urban Gad,jt002
2,tt0002101,Charles L. Gaskill,jt002
3,tt0002130,Francesco Bertolini,jt002
3,tt0002130,Adolfo Padovan,jt002


In [13]:
#Map person name to name DataFrame and add column person_id
new_director_df['person_id'] = new_director_df['person_name'].map(final_name.set_index('person_name')['person_id'])

#Drop person name column to form first part of movie_person_title_junction table
final_director = new_director_df.drop(columns='person_name')
final_director.head()

,movie_id,job_title_id,person_id
0,tt0000574,jt002,nm0846879
1,tt0001892,jt002,NaN
2,tt0002101,jt002,nm0309130
3,tt0002130,jt002,nm0078205
3,tt0002130,jt002,nm0655824


## Writers

In [14]:
# Return only writers and movie title id's
writer_df = imdb_movies[['imdb_title_id', 'writer']].copy()
writer_df.head()

,imdb_title_id,writer
0,tt0000574,Charles Tait
1,tt0001892,"Urban Gad, Gebhard Schätzler-Perasini"
2,tt0002101,Victorien Sardou
3,tt0002130,Dante Alighieri
4,tt0002199,Gene Gauntier


In [15]:
# Rename the column headers
transform_writer_df = writer_df.rename(columns={'imdb_title_id': 'movie_id',
                                                     'writer': 'person_name'})

# Split person_name column by comma and insert it as new row
new_writer_df = transform_writer_df.assign(person_name=transform_writer_df.person_name.str.split(', ')).explode('person_name')

# Add job title id to all rows
new_writer_df['job_title_id'] ='jt003'
new_writer_df.head()

,movie_id,person_name,job_title_id
0,tt0000574,Charles Tait,jt003
1,tt0001892,Urban Gad,jt003
1,tt0001892,Gebhard Schätzler-Perasini,jt003
2,tt0002101,Victorien Sardou,jt003
3,tt0002130,Dante Alighieri,jt003


In [16]:
#Map person name to name DataFrame and add column person_id
new_writer_df['person_id'] = new_writer_df['person_name'].map(final_name.set_index('person_name')['person_id'])

#Drop person name column to form second part of movie_person_title_junction table
final_writer = new_writer_df.drop(columns='person_name')
final_writer.head()

,movie_id,job_title_id,person_id
0,tt0000574,jt003,nm0846879
1,tt0001892,jt003,NaN
1,tt0001892,jt003,NaN
2,tt0002101,jt003,nm0765026
3,tt0002130,jt003,nm0019604


## Actors

In [17]:
# Return only actors and movie title id's
actor_df = imdb_movies[['imdb_title_id', 'actors']].copy()
actor_df.head()

,imdb_title_id,actors
0,tt0000574,"Elizabeth Tait, John Tait, Norman Campbell, Be..."
1,tt0001892,"Asta Nielsen, Valdemar Psilander, Gunnar Helse..."
2,tt0002101,"Helen Gardner, Pearl Sindelar, Miss Fielding, ..."
3,tt0002130,"Salvatore Papa, Arturo Pirovano, Giuseppe de L..."
4,tt0002199,"R. Henderson Bland, Percy Dyer, Gene Gauntier,..."


In [18]:
# Rename the column headers
transform_actor_df = actor_df.rename(columns={'imdb_title_id': 'movie_id',
                                                     'actors': 'person_name'})

# Split person_name column by comma and insert it as new row
new_actor_df = transform_actor_df.assign(person_name=transform_actor_df.person_name.str.split(', ')).explode('person_name')

# Add job title id to all rows
new_actor_df['job_title_id'] ='jt001'
new_actor_df.head()

,movie_id,person_name,job_title_id
0,tt0000574,Elizabeth Tait,jt001
0,tt0000574,John Tait,jt001
0,tt0000574,Norman Campbell,jt001
0,tt0000574,Bella Cola,jt001
0,tt0000574,Will Coyne,jt001


In [19]:
#Map person name to name DataFrame and add column person_id
new_actor_df['person_id'] = new_actor_df['person_name'].map(final_name.set_index('person_name')['person_id'])

#Drop person name column to form last part of movie_person_title_junction table
final_actor = new_actor_df.drop(columns='person_name')
final_actor.head()

,movie_id,job_title_id,person_id
0,tt0000574,jt001,nm0846887
0,tt0000574,jt001,nm0846894
0,tt0000574,jt001,nm3002376
0,tt0000574,jt001,nm0170118
0,tt0000574,jt001,NaN


## movie_person_title_junction table

In [20]:
# Combine director, writer, actor DataFrames
combined_df = pd.concat([final_director, final_writer, final_actor], axis=0)

# Drop rows with no person_id
junction_df = combined_df.dropna(subset=['person_id'])
junction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766171 entries, 0 to 81272
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   movie_id      766171 non-null  object
 1   job_title_id  766171 non-null  object
 2   person_id     766171 non-null  object
dtypes: object(3)
memory usage: 23.4+ MB


In [21]:
final_mpt_junction = junction_df[['movie_id', 'person_id', 'job_title_id']]
final_mpt_junction.head()

,movie_id,person_id,job_title_id
0,tt0000574,nm0846879,jt002
2,tt0002101,nm0309130,jt002
3,tt0002130,nm0078205,jt002
3,tt0002130,nm0655824,jt002
4,tt0002199,nm0646058,jt002


### Create database connection

In [22]:
connection_string = f'{username}:{password}@localhost:5432/movies_db'
engine = create_engine(f'postgresql://{connection_string}')

In [23]:
# Confirm tables
engine.table_names()

['movie', 'movie_person_title_junction', 'person', 'job_title']

### Load DataFrames into database

In [24]:
#Load movie DF into SQL
transform_movie_df.to_sql(name='movie', con=engine, if_exists='append', index=True)

In [25]:
#Load person DF into SQL
final_name.to_sql(name='person', con=engine, if_exists='append', index=False)

In [26]:
#Load job_title DF into SQL
job_title_df.to_sql(name='job_title', con=engine, if_exists='append', index=False)

In [27]:
#Load movie_person_title_junction DF into SQL
final_mpt_junction.to_sql(name='movie_person_title_junction', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_movie_person_title_junction"
DETAIL:  Key (movie_id, person_id, job_title_id)=(tt0021725, nm0782682, jt002) already exists.

[SQL: INSERT INTO movie_person_title_junction (movie_id, person_id, job_title_id) VALUES (%(movie_id)s, %(person_id)s, %(job_title_id)s)]
[parameters: ({'movie_id': 'tt0000574', 'person_id': 'nm0846879', 'job_title_id': 'jt002'}, {'movie_id': 'tt0002101', 'person_id': 'nm0309130', 'job_title_id': 'jt002'}, {'movie_id': 'tt0002130', 'person_id': 'nm0078205', 'job_title_id': 'jt002'}, {'movie_id': 'tt0002130', 'person_id': 'nm0655824', 'job_title_id': 'jt002'}, {'movie_id': 'tt0002199', 'person_id': 'nm0646058', 'job_title_id': 'jt002'}, {'movie_id': 'tt0002423', 'person_id': 'nm0523932', 'job_title_id': 'jt002'}, {'movie_id': 'tt0002461', 'person_id': 'nm0130633', 'job_title_id': 'jt002'}, {'movie_id': 'tt0002461', 'person_id': 'nm0443856', 'job_title_id': 'jt002'}